In [ ]:
# Get the data.
import requests
import pandas as pd

base_url = "https://api.gbif.org/v1/occurrence/search?"

def get_GBIF_response(base_url, offset, params, df):
    """Performs an API call to the base URL with additional parameters listed in 'params'. Concatenates response to a Pandas DataFrame, 'df'."""
    
    #Construct the query URL
    query = base_url+'&'+f'offset={offset}'
    for each in params:
        query = query+'&'+each
    
    #Call API
    response = requests.get(query)
    
    #If call is successful, add data to df
    if response.status_code != 200:
        print(f"API call failed at offset {offset} with a status code of {response.status_code}.")
    else:
        result = response.json()
        df_concat = pd.concat([df, pd.DataFrame.from_dict(result['results'])], axis = 0, ignore_index = True, sort = True)
        endOfRecords = result['endOfRecords']
        return df_concat, endOfRecords, response.status_code
    #set parameters for API call
    params = ['limit=300', 'taxon_key=7681', 'hasCoordinate=true', 
    'hasGeospatialIssue=false']
    #Set up a simple while loop to continue downloading until the last #page
    df = pd.DataFrame()
    endOfRecords = False
    offset = 0
    status = 200
    while endOfRecords == False and status == 200:
        df, endOfRecords, status = get_GBIF_response(base_url, offset, params, df)
        offset = len(df) + 1

In [ ]:
# Save data as a CSV file for cleanup.
df.to_csv(r'C:\Users\Norrie\Palm Trees\Arecaceae.csv', index = False)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 1_Acanthophoenix to Astrocaryum.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Acanthophoenix',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Acoelorraphe',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Acoelorrhaphe',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Acrocomia',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Actinokentia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Adonidia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Aiphanes',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Allagoptera',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Ammandra',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Ancistrophyllum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Aphandra',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Archontophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Areca',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Arecaceae',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Arecastrum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Arenga',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Asterogyne',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Astrocaryum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Acanthophoenix to Astrocaryum)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Acanthophoenix to Astrocaryum) by Norrie')

In [59]:
import plotly.io as pio
pio.write_html(figure, file='map1.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 2_Attalea.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'Attalea',
                 method = 'update',
                 args = [{'visible': [True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title and its location
title = go.layout.Title(text = "Palm Tree World Map (Attalea)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Attalea) by Norrie')

In [61]:
import plotly.io as pio
pio.write_html(figure, file='map2.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 2_Bactris.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'Bactris',
                 method = 'update',
                 args = [{'visible': [True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title and its location
title = go.layout.Title(text = "Palm Tree World Map (Backtris)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Backtris) by Norrie')

In [63]:
import plotly.io as pio
pio.write_html(figure, file='map3.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 3_Balaka to Cocos.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.8,
        xanchor = 'left',
        y = 0.0,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Balaka',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Barcella',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Basselinia',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Beccariophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Bismarckia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Borassodendron',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Borassus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Brahea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Brassiophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Burretiokentia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Butia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Calamus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Calyptrocalyx',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Calyptrogyne',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Calyptronoma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Carpentaria',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Caryota',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Ceroxylon',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Chamaedorea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Chamaedoreaceae',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Chamaedrea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Chamaerops',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Chelyocarpus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Chrysalidocarpus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Chuniophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Clinosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Coccothrinax',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Cocos',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Balaka to Cocos)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Balaka to Cocos) by Norrie')

In [65]:
import plotly.io as pio
pio.write_html(figure, file='map4.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 4_Colpothrinax to Gaussia.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Colpothrinax',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Copernicia',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Corypha',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Cryosophila',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Cyrtostachys',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Daemonorops',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Deckenia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Desmoncus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Dictyocaryum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Dictyosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Drymophloeus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Dypsis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Elaeis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Eleiodoxa',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Eremospatha',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Erythea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Eugeissona',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Euterpe',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Gaussia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Colpothrinax to Gaussia)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Colpothrinax to Gaussia) by Norrie')

In [67]:
import plotly.io as pio
pio.write_html(figure, file='map5.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 5_Guihaia to Nypa.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.8,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Guihaia',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Gulubia',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hedyscepe',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hemithrinax',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Heterospathe',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Howea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hydriastele',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hyophorbe',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hyospathe',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Hyphaene',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Iguanura',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Iriartea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Iriartella',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Itaya',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Jailoloa',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Johannesteijsmannia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Juania',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Jubaea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Jubaeopsis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Kentia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Kerriodoxa',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Korthalsia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Laccospadix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Laccosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lanonia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Latania',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lemurophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Leopoldinia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lepidocaryum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lepidorrhachis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Leucothrinax',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Licuala',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Linospadix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Livistona',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lodoicea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Lytocaryum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Manicaria',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Marojejya',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Masoala',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Mauritia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Mauritiella',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Medemia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Metroxylon',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Myrialepis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Nannorrhops',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Nenga',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Neonicholsonia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Nephrosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Normanbya',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Nypa',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Guihaia to Nypa)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Guihaia to Nypa) by Norrie')

In [69]:
import plotly.io as pio
pio.write_html(figure, file='map6.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 6_Oenocarpus to Rhapis.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.8,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Oenocarpus',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Oncocalamus',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Oncosperma',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Orania',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Oraniopsis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Palmae',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Parajubaea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Phoenicophorium',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Phoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Pholidostachys',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Physokentia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Phytelephas',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Pigafetta',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Pinanga',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Plectocomia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Plectocomiopsis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Podococcus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Polyandrococos',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Prestoea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Pritchardia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Pseudophoenix',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Ptychococcus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Ptychosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Raphia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Ravenea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Reinhardtia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Rhapidophyllum',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Rhapis',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Oenocarpus to Rhapis)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Oenocarpus to Rhapis) by Norrie')

In [71]:
import plotly.io as pio
pio.write_html(figure, file='map7.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 7_Rhopaloblaste to Syagrus.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Rhopaloblaste',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Rhopalostylis',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Roscheria',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Roystonea',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Sabal',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Sabinaria',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Salacca',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Saribus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Satranala',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Scheelea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Schippia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Sclerosperma',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Serenoa',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Socratea',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Solfia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Sommieria',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Syagrus',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Rhopaloblaste to Syagrus)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Rhopaloblaste to Syagrus')

In [73]:
import plotly.io as pio
pio.write_html(figure, file='map8.html', auto_open=True)

In [ ]:
# Import Plotly to map the data with mapbox tools (create Username, API key, and Access Token at Plotly and mapbox following the aforementioned tutorial by Emma).
import pandas as pd
import chart_studio as py
import plotly.graph_objs as go
from chart_studio.plotly import iplot
import plotly
import pandas as pd

# Setting user, api key and access token
py.tools.set_credentials_file(username='NorrieF', api_key='jVf7ynJyxiVAHGsmfjtS')
mapbox_access_token = 'pk.eyJ1Ijoibm9ycmllZiIsImEiOiJjazlkanB6czIwNGZ3M3BzNjdrMXFlemo1In0.S3Nd5-tLrwK7hLTffw3vTw'

# Load the cleaned up version of Arecaceae.csv
df = pd.read_csv(r"C:\Users\Norrie\Palm Trees\Map Data 8_Synechanthus to Wodyetia.csv", encoding='latin-1') 

# Create a list of species from the unique values of the column "Name" in the dataframe
species = df.Name.unique()

# Define the data to show.
data = []
for i in species:
    palm_data = dict(
            lat = df.loc[df['Name'] == i,'Latitude'],
            lon = df.loc[df['Name'] == i,'Longitude'],
            name = i,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(palm_data)

# Lay out the map.
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 35,
            lon = -140
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 1,
        # default map style
        style = 'dark'
    )
)

# Define the contents of the map.
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.985,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select species of palm trees to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All Species',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True, True, True]}]),
            dict(label = 'Synechanthus',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Tahina',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Thrinax',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Trachycarpus',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Trithrinax',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Veitchia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = 'Verschaffeltia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False, False, False]}]),
            dict(label = 'Voanioala',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = 'Wallichia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = 'Washingtonia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = 'Welfia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = 'Wendlandiella',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = 'Wettinia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = 'Wodyetia',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False, False, True]}]),
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.00,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# Assign the list of dictionaries to the layout dictionary.
layout['updatemenus'] = updatemenus

# Decide the title
title = go.layout.Title(text = "Palm Tree World Map (Species: Synechanthus to Wodyetia)", x = 0.5)
# Add the title.
layout['title'] = title

# Generate the map.
figure = dict(data = data, layout = layout)
iplot(figure, filename = 'Palm Tree World Map (Species: Synechanthus to Wodyetia) by Norrie')

In [76]:
import plotly.io as pio
pio.write_html(figure, file='map9.html', auto_open=True)